---
### <center> Carga de Datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Carga y visualización parcial de Datos:
df = pd.read_csv('datasets/generacin-histrica.csv')
df.head(5)

---
### <center> Preparación de Datos

In [ ]:
df.info()

In [ ]:
# Chequeo de registros duplicados:
df.duplicated().value_counts()

Observamos que el DataFrame posee 91543 registros, y que cada columna posee la misma cantidad de objetos no nulos (ni *'Nan'* ni *'None'*) a excepción de *'categoria_hidraulica'*, aunque esta es una columna no significativa para nuestro análisis. Además, no existen registros duplicados.

In [ ]:
# Eliminamos la columna 'categoria_hidraulica':
df.drop(['categoria_hidraulica'], axis=1, inplace=True)

In [ ]:
# Chequeo de países presentes:
df['pais'].value_counts()

In [ ]:
# Redefinimos el dataset, conservando sólamente los registros válidos para Argentina:
df = df[df['pais'] == "Argentina"]
df['pais'].value_counts()

In [ ]:
# Como hemos eliminado registros, reseteamos los índices del DataFrame:
df.set_index((i for i in range(len(df))), inplace=True)

In [ ]:
# Información actualizada:
df.info()

El DataFrame consta de 91541 registros, sin inconsistencias, valores no nulos ni duplicados. Todos los registros son pertenecientes a la Argentina.

---
### <center> Análisis exploratorio de Datos

Al consultar las **'fuentes_generacion'**, observamos que existen varias clasificaciones para las tecnologías no renovables (Térmica, Hidráulica y Nuclear):

In [ ]:
# Convertir la variable categórica 'fuente_generacion' en variables dummy
#df = pd.get_dummies(df, columns=['fuente_generacion'], drop_first=True)


In [ ]:
#df['fuente_generacion'].value_counts()

In [ ]:
# Analizamos los Tipos de tecnología disponibles:
df['tecnologia'].value_counts()

Estas Tecnologías se corresponden con las siguientes descripciones:

**No Renovables:**

* DI: Diesel
* TG: Turbo Gas
* CC: Ciclos Combinados
* HID: Hidroeléctrica
* NUC: Nuclear


**Renovables:**

* HR: Resto Hidráulico
* TV: Turbo Vapor
* EOL: Eólica
* SOL: Solar
* BIOM: Biomasa
* BIOG: Biogas
* MHID: Microhidráulica

In [ ]:
# Corroboramos esta clasificación, consultando aquellas consideradas 'Renovables':
df[df['fuente_generacion'] == "Renovable"]['tecnologia'].value_counts()

Como la columna **'fuente_generacion'** posee varias clasificaciones según el origen de las fuentes no renovables, agrupamos estas clasificaciones y agregamos una columna **'renovable'** con valores booleanos que indique claramente si la producción es renovable o no:

In [ ]:
# Creamos los datos para la columna 'renovable':
col_renovables = []

for i in range(len(df)):
    no_renovables = ["DI", "TG", "CC", "HID", "NUC"]
    estado = True
    if df['tecnologia'][i] in no_renovables:
        estado = False
    col_renovables.append(estado)

# La asignamos al DataFrame:
df['renovable'] = col_renovables

Consultamos la producción energética anual según si es renovable o no:

In [ ]:
# Consultamos la generación neta en MWh según año y según origen renovable:
anual_MWh = df.groupby(['anio', 'renovable'])['generacion_neta_MWh'].sum()

# Convertimos los valores a TWh (Terawatts por hora) para una mejor lectura:
anual_TWh = anual_MWh / 1000000
anual_TWh

In [ ]:
# Generamos un gráfico de barras apiladas:
anual_TWh.unstack().plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Generación eléctrica total anual según Renovabilidad")
plt.xlabel("Año")
plt.ylabel("Generación (TWh)")
plt.show()